In [ ]:
# pip install pymysql

In [ ]:
# pip install sqlalchemy

In [ ]:
# pip install sqlalchemy_utils

In [8]:
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import MetaData
import getpass
password = getpass.getpass()
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

function to format to datetime; dropping duplicates

In [9]:
def date_formatting (df, col):
    df[col] = df[col].astype("datetime64[ns]")
    df["year"] = pd.DatetimeIndex(df[col]).year
    df = df.sort_values(by=col, ascending = 0)
    df = df.drop_duplicates(subset = [col])
    return df

function to apply snake_case

In [10]:
def snake_case(df):
    cols=[]
    for i in df.columns:
        cols.append(i.lower())
    df.columns = cols
    df.columns = df.columns.str.replace(' ', '_')
    return df

read all downloaded and scraped csv (code script for scraping: see "PART 1: web scraping")

In [11]:
gold = pd.read_csv("./files/gold price.csv")

silver = pd.read_csv("./files/silver price.csv")

platinum = pd.read_csv("./files/platinum price.csv")
platinum = platinum[["Date","Close"]]
platinum = platinum.rename(columns = {"Close":"Price"})

palladium = pd.read_csv("./files/palladium price.csv")
palladium = palladium[["Date","Price"]]

coal_monthly = pd.read_csv("./files/coal price monthly.csv")
coal_monthly = coal_monthly.rename(columns = {"DATE":"Date", "PCOALAUUSDM":"Price"})

n_gas_1 = pd.read_csv("./files/natural gas price 1.csv")
n_gas_1 = n_gas_1[["Date","Price"]]
n_gas_2 = pd.read_csv("./files/natural gas price 2.csv")
n_gas_2 = n_gas_2[["Date","Price"]]
n_gas = [n_gas_1, n_gas_2]
n_gas = pd.concat(n_gas)

oil_1 = pd.read_csv("./files/crude oil price 1.csv")
oil_1 = oil_1[["Date","Price"]]
oil_2 = pd.read_csv("./files/crude oil price 2.csv")
oil_2 = oil_2[["Date","Price"]]
oil = [oil_1, oil_2]
oil = pd.concat(oil)

euro_1 = pd.read_csv("./files/eur price 1.csv")
euro_1 = euro_1[["Date","Price"]]
euro_2 = pd.read_csv("./files/eur price 2.csv")
euro_2 = euro_2[["Date","Price"]]
euro = [euro_1, euro_2]
euro = pd.concat(euro)

gbp_1 = pd.read_csv("./files/gbp price 1.csv")
gbp_1 = gbp_1[["Date","Price"]]
gbp_2 = pd.read_csv("./files/gbp price 2.csv")
gbp_2 = gbp_2[["Date","Price"]]
gbp = [gbp_1, gbp_2]
gbp = pd.concat(gbp)

dj_ind = pd.read_csv("./files/dow jones industrial stock price.csv")

dj_trans = pd.read_csv("./files/dow jones tansportation stock price.csv")

nasdaq = pd.read_csv("./files/nasdaq stock price.csv")

snp = pd.read_csv("./files/snp stock price.csv")

scco = pd.read_csv("./files/southern copper corp price.csv")
scco = scco[["Date","Close"]]
scco = scco.rename(columns = {"Close":"Price"})

rio = pd.read_csv("./files/rio tinto group price.csv")
rio = rio[["Date","Close"]]
rio = rio.rename(columns = {"Close":"Price"})

royalg = pd.read_csv("./files/royal gold price.csv")
royalg = royalg[["Date","Close"]]
royalg = royalg.rename(columns = {"Close":"Price"})

newmont = pd.read_csv("./files/newmont corp price.csv")
newmont = newmont[["Date","Close"]]
newmont = newmont.rename(columns = {"Close":"Price"})

agnico = pd.read_csv("./files/agnico eagle mines price.csv")
agnico = agnico[["Date","Close"]]
agnico = agnico.rename(columns = {"Close":"Price"})

cleaning up the columns and apply datetime type

In [12]:
dataframes = [gold, silver, platinum, palladium, coal_monthly, n_gas, oil, euro, gbp, dj_ind, dj_trans, nasdaq, snp, scco, rio, royalg, newmont, agnico]
for dataframe in dataframes:
    dataframe = snake_case(dataframe)
    dataframe = date_formatting(dataframe, "date")
    display(dataframe.shape)

(6707, 3)

(6668, 3)

(5114, 3)

(2320, 3)

(312, 3)

...

(5285, 3)

(5285, 3)

(5285, 3)

(5285, 3)

(5285, 3)

create connection to mysql, then make a new SQL database with name "mid_project" if it doesn't exist yet.

In [16]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/mid_project2'
engine = create_engine(connection_string)
if not database_exists(engine.url):
    create_database(engine.url)

metadata is like temporary memory in python to collect data (or dataframes) that we want to put in the SQL database

UPDATE: metadata not needed here, hehe

In [ ]:
# metadata_obj = MetaData()

making tables in SQL database out of pandas dataframe

In [17]:
gold.to_sql("gold", engine, if_exists = "replace")
silver.to_sql("silver", engine, if_exists = "replace")
platinum.to_sql("platinum", engine, if_exists = "replace")
palladium.to_sql("palladium", engine, if_exists = "replace")
# coal.to_sql("coal", engine, if_exists = "replace")
coal_monthly.to_sql("coal_mnthl", engine, if_exists = "replace")
n_gas.to_sql("n_gas", engine, if_exists = "replace")
oil.to_sql("oil", engine, if_exists = "replace")
euro.to_sql("euro", engine, if_exists = "replace")
gbp.to_sql("gbp", engine, if_exists = "replace")
dj_ind.to_sql("dowjones_ind", engine, if_exists = "replace")
dj_trans.to_sql("dowjones_transp", engine, if_exists = "replace")
nasdaq.to_sql("nasdaq", engine, if_exists = "replace")
snp.to_sql("snp", engine, if_exists = "replace")
scco.to_sql("scco", engine, if_exists = "replace")
rio.to_sql("rio", engine, if_exists = "replace")
royalg.to_sql("royal_gold", engine, if_exists = "replace")
newmont.to_sql("newmont", engine, if_exists = "replace")
agnico.to_sql("agnico", engine, if_exists = "replace")

5285

make query to JOIN all the tables with rows with the same date

In [ ]:
query = '''
select distinct euro.date, euro.year, gold.price as gold,
silver.price as silver, platinum.price as platinum,
palladium.price as palladium, coal_mnthl.price as coal_monthly,
n_gas.price as n_gas, oil.price as oil,
euro.price as euro, gbp.price as gb_pound,
dowjones_ind.price as dowjones_industrial, dowjones_transp.price 
as dowjones_transport, nasdaq.price as nasdaq, snp.price as snp,
scco.price as southern_copper, rio.price as rio_tinto,
royal_gold.price as royal_gold, newmont.price as newmont,
agnico.price as agnico_mines
from euro left join gbp on euro.date = gbp.date
left join gold on euro.date = gold.date
left join silver on euro.date = silver.date
left join oil on euro.date = oil.date
left join n_gas on euro.date = n_gas.date
left join platinum on euro.date = platinum.date
left join palladium on euro.date = palladium.date
left join coal_mnthl on euro.date = coal_mnthl.date
left join dowjones_ind on euro.date = dowjones_ind.date
left join dowjones_transp on euro.date = dowjones_transp.date
left join nasdaq on euro.date = nasdaq.date
left join snp on euro.date = snp.date
left join scco on euro.date = scco.date
left join rio on euro.date = rio.date
left join royal_gold on euro.date = royal_gold.date
left join newmont on euro.date = newmont.date
left join agnico on euro.date = agnico.date
order by euro.date asc;
'''
data = pd.read_sql_query(query, engine)
data.shape

save to "data_dirty.csv"

In [ ]:
data.to_csv("data_dirty.csv")

In [ ]:
df.index1[df["country"] == 1]